In [ ]:
import requests
import json
import pandas as pd
from time import sleep

In [ ]:
df = pd.read_csv('../output/codici_umbria_er.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Umbria
df_er = df_comuni[df_comuni['cod_regione'] == '10']

In [ ]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

In [ ]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [ ]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
  response = requests.get(url, headers=headers)
  data = response.json()
  
  comune = data['int']['desc_com']
  provincia = data['int']['desc_prov']  
  regione = df_er.iloc[idx]['regione']  # Keep from df_er if not in response
  url = df_er.iloc[idx]['url']

  print(f'getting {comune}, {provincia}')

  # Initialize comune_info with basic location details
  comune_info = {
      'comune': comune,
      'provincia': provincia,
      'regione': regione,
      'url': url
  }
  
  # Track unique parties and candidate data
  if 'cand' in data:
      # First, collect party information
      for cand in data['cand']:
          if 'liste' in cand:
              for lista in cand['liste']:
                  party_name = lista['desc_lis_c']
                  comune_info[party_name] = lista['voti']
                  comune_info[f"{party_name}_perc"] = lista['perc']
      
      # Then, add candidate information
      for cand in data['cand']:
          nome_completo = f"{cand['nome']} {cand['cogn']}"
          comune_info[f"{nome_completo}_total_votes"] = cand['voti']
          comune_info[f"{nome_completo}_total_perc"] = cand['perc']

  comuni_data.append(comune_info)
  
  sleep(0.1)

# Create DataFrame
df_data = pd.DataFrame(comuni_data)

In [ ]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data.head(2)

In [ ]:
df_data.to_csv('../output/risultati_UM.csv', index=False, encoding='UTF-8-sig')

### Formatting for Datawrapper visualization

In [ ]:
df_data = pd.read_csv('../output/risultati_um.csv')

In [ ]:
candidates = df_data[['comune', 'provincia', 'regione', 'STEFANIA PROIETTI_total_votes', 'STEFANIA PROIETTI_total_perc', 'DONATELLA TESEI_total_votes', 'DONATELLA TESEI_total_perc']]

candidates.head(2)

In [ ]:
# Create a copy of the DataFrame
candidates = candidates.copy()

# Basic cleaning for Datawrapper integration
candidates['comune'] = candidates['comune'].str.replace("Citta'", "Città", regex=False)

In [ ]:
# Calculate the left-right index
candidates['left_right_index'] = (candidates['STEFANIA PROIETTI_total_votes'] - candidates['DONATELLA TESEI_total_votes']) / (candidates['STEFANIA PROIETTI_total_votes'] + candidates['DONATELLA TESEI_total_votes'])
candidates.sample(5)

In [ ]:
candidates.to_csv('../output/viz/ris_candidati_UM.csv', index=False, encoding='UTF-8-sig')